In [1]:
import tensorflow as tf
from tensorflow import keras

# # Define the neural network architecture
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(10, input_shape=(1,), activation='relu'),
#     tf.keras.layers.Dense(1)
#    ])
# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')

# ==============================================================
# Time series prediction model  / DNN architecture function
model = keras.Sequential([
keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(1,)),
    keras.layers.Dense(2, activation=tf.nn.relu),
    keras.layers.Dense(1)])

optimizer = tf.keras.optimizers.Adam()
model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
# ==============================================================


import wave
import numpy as np

# table of index
IndexTable = [-1, -1, -1, -1, 2, 4, 6, 8, -1, -1, -1, -1, 2, 4, 6, 8]

# table of  quantizer step size
StepSizeTable = [7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 19, 21, 23, 25, 28, 31, 34, 37, 41, 45, 50, 55, 60, 66, 73, 80, 88, 97, 107, 118, 130, 143, 157, 173, 190, 209, 230, 253, 279, 307, 337, 371, 408, 449, 494, 544, 598, 658, 724, 796, 876, 963, 1060, 1166, 1282, 1411, 1552, 1707, 1878, 2066, 2272, 2499, 2749, 3024, 3327, 3660, 4026, 4428, 4871, 5358, 5894, 6484, 7132, 7845, 8630, 9493, 10442, 11487, 12635, 13899, 15289, 16818, 18500, 20350, 22385, 24623, 27086, 29794, 32767]


predsample = 0
index = 0

def ADPCM_Encode(sample):
    global index
    global predsample
    
    global diffq
    global ore_diff
    global error_signal
    
    code = 0
    
    step_size = StepSizeTable[index]

    # Use the neural network to predict the difference
    #prediction = model.predict(predsample)
    prediction = model.predict(np.array([predsample]))

    # calculate difference
    diff = sample - predsample + prediction
    
    if diff < 0:
        code = 8
        diff = -diff
    # quantize the diff into ADPCM code
    # inverse quantize the code into a predicted diff
    tmpstep = step_size
    diffq = step_size >> 3
    if diff >= tmpstep:
        code = code | 0x04
        diff -= tmpstep
        diffq = diffq + step_size
    tmpstep = tmpstep >> 1
    if diff >= tmpstep:
        code = code | 0x02
        diff = diff - tmpstep
        diffq = diffq + (step_size >> 1)
    tmpstep = tmpstep >> 1
    if diff >= tmpstep:
        code = code | 0x01
        diffq = diffq + (step_size >> 2)
        
    # fixed predictor to get new predicted sample
    if code & 8:
        predsample = predsample - diffq
    else:
        predsample = predsample + diffq
        
    # check for overflow
    if predsample > 32767:
        predsample = 32767
    elif predsample < -32768:
        predsample = -32768
        
    # find new stepsize index
    index += IndexTable[code]
    
    # check for overflow
    if index < 0:
        index = 0
    if index > 88:
        index = 88
    
    # update the model's parameters based on the prediction error
         
    model.fit(np.array([predsample]), diff, epochs=1, verbose=0)
    
    # return new ADPCM code code & 0x0f == code and predsample
    return code & 0x0f, predsample

In [ ]:
import matplotlib.pyplot as plt
#%pmatplotlib inline
import IPython.display as ipd
import numpy as np

# ...............scipy wave based data..............
import wave
from scipy.io import wavfile
np.set_printoptions(threshold=np.inf)
audio_file = 'audio_data/Test_dataset_All_4_speakers_merged_MS_and_FS.wav'
fs, data = wavfile.read(audio_file)
ore_data=data[:]

# ================================================================
org_samp=[]
pre_samp=[]
oreginal_diff=[]
adpcm_code = []
q_diff=[]
err_signal=[]

for i in range(len(ore_data)):
    
    ADPCM_sample,pred = ADPCM_Encode(ore_data[i]) 
    print("Sample:", i)
    #print('................................')
    
    org_samp.append(ore_data[i])
    pre_samp.append(pred)
        
org_samp=np.array(org_samp)           #oreginal signal samples
pre_samp=np.array(pre_samp)           #predicted sample 

oreginal_diff=np.array(oreginal_diff) #the oreginal and predicted sample difference
adpcm_code = np.array(adpcm_code)     #the ADPCM code the is transmited to decoder 

In [ ]:
org_samp[:30]

In [ ]:
pre_samp[:30]

In [ ]:
# Normalize Data Between 0 and 1
y = org_samp
ypn = pre_samp

org_samp = np.array(y)
# goal : range [0, 1]
org_samp = (org_samp - min(org_samp)) / ( max(org_samp) - min(org_samp) )

pre_samp = np.array(ypn)
pre_samp = (pre_samp - min(pre_samp)) / ( max(pre_samp) - min(pre_samp) )

In [ ]:
#====================This is a one step forecast (based on that we have constructed the model)====================
plt.figure(figsize=(12, 4))
plt.plot(org_samp[:], label = 'Original speech signal')
plt.plot(pre_samp[:], label = 'Predicted speech signal')
plt.legend()

In [ ]:
error_signal=org_samp-pre_samp

In [ ]:
#====================kjjmmmmmmmmmmmmmmmmmm======
plt.figure(figsize=(12, 4))

# Oreginal speech signal data
plt.title('The Original PCM and predicted Speech Signal') 
plt.xlabel('Number of samples')
plt.ylabel('Amplitude (Quantization level)')
plt.plot(org_samp, 'b', label='Original Speech Signal ')
plt.plot(pre_samp, 'g', label='predicted Speech Signal')
plt.plot(error_signal, 'r', label='Error Speech Signal')
plt.legend(loc='upper center')

In [ ]:
#====================kjjmmmmmmmmmmmmmmmmmm======
plt.figure(figsize=(12, 4))

# Oreginal speech signal data
plt.title('The Original PCM and predicted Speech Signal it was randomly taken from 3000th to 3050th') 
plt.xlabel('Number of samples')
plt.ylabel('Amplitude (Quantization level)')
plt.plot(org_samp[3000:3050], 'b', label='Original Speech Signal ')
plt.plot(pre_samp[3000:3050], 'g', label='predicted Speech Signal')

plt.legend(loc='upper center')

In [ ]:
plt.title('The Original PCM and predicted Speech Signal it was randomly taken from 400000th to 400050th') 


In [ ]:
#====================kjjmmmmmmmmmmmmmmmmmm======
plt.figure(figsize=(12, 4))

# Oreginal speech signal data
plt.title('The Original PCM and predicted Speech Signal it was randomly taken from 400000th to 400050th') 
plt.xlabel('Number of samples')
plt.ylabel('Amplitude (Quantization level)')
plt.plot(org_samp[400000:400050], 'b', label='Original Speech Signal ')
plt.plot(pre_samp[400000:400050], 'g', label='predicted Speech Signal')

plt.legend(loc='upper center')

In [ ]:
#====================kjjmmmmmmmmmmmmmmmmmm======
plt.figure(figsize=(12, 4))

# Oreginal speech signal data
plt.title('The Original PCM and predicted Speech Signal it was randomly taken from 700,000th to 700,050th') 
plt.xlabel('Number of samples')
plt.ylabel('Amplitude (Quantization level)')
plt.plot(org_samp[700000:700050], 'b', label='Original Speech Signal ')
plt.plot(pre_samp[700000:700050], 'g', label='predicted Speech Signal')

plt.legend(loc='upper center')

In [ ]:
# ============= Calculating Signal to Noise Ratio (SNR)===========

error = org_samp[:] - pre_samp[:]

plt.figure(figsize=(12, 4))
plt.plot(error, 'b')
plt.show()

org_sigpow = 10*np.log10(np.mean(pow(np.abs(org_samp[:]),2)))
error_mse = 10*np.log10(np.mean(pow(np.abs(error),2)))

#print(dnn_mse, dnn_sigpow, lms_mse, lms_sigpow)
print("NN based ADPCM predictor SNR:", org_sigpow - error_mse, "dB")